# Clasificación de textos con 🤗 Transformers
Cómo afinar un modelo de RoBERTa en español para clasificar las reseñas de un hotel ubicado en Punta Cana.

# Configuración
Si está ejecutando este notebook en Google Colab, ejecute la siguiente celda para instalar las bibliotecas que necesitamos:

In [ ]:
#!pip install numpy==1.17
!pip install transformers datasets
#!pip install --upgrade --force-reinstall pandas
#!pip install --upgrade tokenizers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
# Esto sólo funciona en Google Colab! Para los notebooks normales, es necesario ejecutar esto en el terminal
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `GoogleColab` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `Google

In [ ]:
# !huggingface-cli logout

# Cargar y explorar los datos
Utilizaremos 🤗 Datasets para cargar y procesar nuestro conjunto de datos.
Como el dataset de las reseñas de amazon ya no es accesible, emplearemos otro de reseñas de un hotel en Punta Cana "beltrewilton/punta-cana-spanish-reviews"


Este es un **ejemplo** para extraer datos de Kaggle, y luego descomprimir y leer el fichero csv como un dataframe de pandas

In [ ]:
'''
%%bash
# Descarga desde Kaggle
# Subir archivo `kaggle.json`
pip install -q kaggle
#mkdir ~/.kaggle
#cp kaggle.json ~/.kaggle/
#chmod 600 ~/.kaggle/kaggle.json
kaggle datasets download -d smunoz3801/9325-letras-de-rap-en-espaol

# mostrar el directorio
!ls /content

import zipfile

zip_file = '/content/9325-letras-de-rap-en-espaol.zip'
extract_path = '/content/datos_extraidos'

# Descomprimir
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

ruta= extract_path + "/hhgroups_merge_28_05.csv"
data = pd.read_csv(ruta)
data

'''

'\n%%bash\n# Descarga desde Kaggle\n# Subir archivo `kaggle.json`\npip install -q kaggle\n#mkdir ~/.kaggle\n#cp kaggle.json ~/.kaggle/\n#chmod 600 ~/.kaggle/kaggle.json\nkaggle datasets download -d smunoz3801/9325-letras-de-rap-en-espaol\n\n# mostrar el directorio\n!ls /content\n\nimport zipfile\n\nzip_file = \'/content/9325-letras-de-rap-en-espaol.zip\'\nextract_path = \'/content/datos_extraidos\'\n\n# Descomprimir\nwith zipfile.ZipFile(zip_file, \'r\') as zip_ref:\n    zip_ref.extractall(extract_path)\n\nruta= extract_path + "/hhgroups_merge_28_05.csv"\ndata = pd.read_csv(ruta)\ndata\n\n'

Dataset de reseñas de hotel

In [ ]:
import pandas as pd
from datasets import load_dataset

#dataset = load_dataset("amazon_reviews_multi", "es") # No es accesible
dataset = load_dataset("beltrewilton/punta-cana-spanish-reviews", "default")

dataset

#print(type(dataset)) <class 'datasets.dataset_dict.DatasetDict'>

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/184 [00:00<?, ?B/s]

punta_cana.csv:   0%|          | 0.00/18.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/34561 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['hotel_name', 'location', 'wrote', 'rating', 'title', 'review_text'],
        num_rows: 34561
    })
})

In [ ]:
import random
import pandas as pd
from datasets import ClassLabel
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    #"Taken from https://github.com/huggingface/notebooks/blob/master/examples/text_classification.ipynb"

    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

show_random_elements(dataset["train"])

,hotel_name,location,wrote,rating,title,review_text
0,The Westin Puntacana Resort & Club,"Montevideo, Uruguay",June 2016,5,Increíble,"Hemos pasado unas vacaciones increíbles. El personal es amable, las instalaciones impecables, la arena de la playa es simplemente espectacular. Lo único que nos ha molestado ha sido la cantidad de insectos. Desayunar a veces es molesto con la cantidad de insectos que hay. Pocos locales conocen la propiedad, por lo que desplazarse desde Bávaro hacía el resort puede ser algo agotador. Por lo demás un muy sitio para descansar, la playa es muy tranquila."
1,Hotel Punta Cana Princess All Suites Resort & Spa,"Madrid, Spain",December 2019,4,Relax,"Me gustó mucho!! El hotel muy bonito por fuera, muy cuidado y acogedor!! Los restaurantes temáticos buenísimos, comida de calidad y el buffet tenía lo típico que nunca cambia en comida y q a cualquiera le puede gustar (ensaladas, hamburguesas, pasta, arroces) y cada día ponían comidas distintas, guisos de pescados, guisos de carnes, con lo cual había variedad. Además de esto, por las noches cada noche lo hacían temático (una noche comida mexicana, otra americana, asiática....). Pero lo que sin duda tengo que resaltar son sus trabajadores, su equipo de animación! Me vine con pena porque al terminar la semana parecían como familia... tengo q nombrar en concreto por cómo nos sentimos cuidados, mimados, animados, etc.... a FRANCIS FELICIANO (que a pesar de q al principio me resultó"
2,Caribe Deluxe Princess,"San Carlos de Bariloche, Argentina",October 2019,1,pesimo lugar NO VAYAN,"Lamentablemente luego de haber asisitido muchas veces a hoteles de la cadena Royal Holiday sin inconvenientes acabo de regresar de una pesima experiencia en este hotel. Lo mas importante el hotel estaba INFECTADO DE PULGAS varios de mis compañeros de viaje hemos regresados con nuestras piernas lastimadas por esta plaga, ya estoy en tratamiento medico con corticoides y antibiotico. Segundo MUY FLOJA la calidad de la comida y pesima la calidad basica de las bebidas, para acceder a un nivel mejor habia que pagar US25 por dia. El personal es muy buena gente pero no puedo dejar de decir la verdad de la mala experiencia que he tenido. Tambien las habitaciones estaban muy mal mantenidas Agradecimiento especial a la mucama ENEROLISA su nombre que atendio con esmero y dedicacion"
3,Paradisus Punta Cana Resort,"Lapuebla de Labarca, Spain",August 2018,5,Segundo año consecutivo....,"Hola me llamo Mariana,soy de Argentina y estuve alojada en Paradisus Punta Cana del 2 al 15/8....el motivo del viaje fue de vacaciones...y que vacaciones....!!! fuimos con Roberto,mi esposo,por segundo año consecutivo,que mas decirles....desde las chicas de la recepcion de Royal service,pasando por mi queridisma y estimada Maria Rondon(la cual estuvo con nosotros el año pasado tambieny volveremos a pedir el proximo año) todos pero absolutamente todos los chicos que atiendenen el restaurante lapalapa,Kintina,Billis(UN GENIO),Lupe,y todos todos,los chicos del bar de playa,Maximo,Miguel,Ranieri....habia otros doschicos mas que nome acuerdo el nombre y pido discupas,los chicos del restaurante fuego,Maria Donastorg que con su amabilidad nos recibia,la ama de llaves,Yajaina de"
4,Grand Palladium Bavaro Suites Resort & Spa,Dominican Republic,February 2020,1,Hotel cutre,"Cinco parejas con sus respectivos hijos fuimos a pasar una semana en este hotel, conclusión, cuatro parejas tuvieron que pedir que les cambiaran la habitación porque no les funcionaba nada.Mi pareja y yo no pedimos el cambio porque veíamos que las habitaciones que les habían ofrecido a nuestros amigos, eran iguales o peores, asi que decidimos quedarnos en la misma y avisamos a limpiadoras y recepción que arreglaran cosas como la alcachofa de la ducha, la cual nunca fue arreglada , el tapon del jacuzzi, que nunca fue arreglado y jamas pudimos volver a utilizar,y las goteras del aire acondicionado.Las habitaciones están para tirarlas y hacerlas de nuevo, no se co

In [ ]:
# se pasa el DatasetDict a DataFrame para poderlo manejar mejor
dataset.set_format("pandas")
df = dataset["train"][:]
df.head()
#print(df.columns) Index(['hotel_name', 'location', 'wrote', 'rating', 'title', 'review_text'], dtype='object')

,hotel_name,location,wrote,rating,title,review_text
0,Secrets Cap Cana Resort & Spa,"Villaviciosa, Spain",December 2021,5,Vacaciones en El paraíso,El hotel impecable todo muy limpio las habitac...
1,Secrets Cap Cana Resort & Spa,"Villaviciosa, Spain",December 2021,5,HOTEL de la EXCELENCIA,"EXCELENTE, magnífico, excepcional, maravilloso..."
2,Secrets Cap Cana Resort & Spa,"Villaviciosa, Spain",December 2021,5,Sentirse como en CASA!!!!!,Tras conocer bastantes hoteles en distintos lu...
3,Secrets Cap Cana Resort & Spa,"Villaviciosa, Spain",December 2021,5,Muy buen nivel,Muy buen hotel. Cumple con todos los requisito...
4,Secrets Cap Cana Resort & Spa,"Villaviciosa, Spain",December 2021,5,10,Excelente todo ....sobre todo el personal. Es ...


In [ ]:
df["rating"].value_counts()

,count
rating,
5,26708
4,3924
3,1695
1,1334
2,900


# Fusionar las clasificaciones por rating

In [ ]:
# Se realiza desde el dataframe y NO desde el dataset
# Se eliminan los datos de valor 3 para quedarnos con los extremos
df_sin_indice = df.reset_index(drop=True)
df_sin_indice = df_sin_indice.loc[df_sin_indice["rating"] != 3]

#df_sin_indice["rating"].value_counts()


In [ ]:
# Se crea nuevo campo labels para identificar las buenas reseñas (1) y malas reseñas (0)
df_sin_indice["labels"] = df_sin_indice["rating"].map(lambda x: 0 if x <= 2 else 1)

In [ ]:
df_sin_indice["labels"].value_counts()

,count
labels,
1,30632
0,2234


# Tokenizar las reseñas

In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "BSC-TeMU/roberta-base-bne"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.15M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/509k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.46M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

# Tokenización de las reseñas


In [ ]:
tokenizer.vocab_size

50262

In [ ]:
text = "¡hola, me llamo ejemplo!"
tokenized_text = tokenizer.encode(text)

for token in tokenized_text:
    print(token, tokenizer.decode([token]))

0 <s>
1465 ¡
12616 hola
66 ,
503  me
17111  llamo
1915  ejemplo
55 !
2 </s>


In [ ]:
encoded_text = tokenizer(text, return_tensors="pt")
encoded_text

{'input_ids': tensor([[    0,  1465, 12616,    66,   503, 17111,  1915,    55,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
def tokenize_reviews(examples):
    #return tokenizer(examples["review_text"], truncation=True)
    return tokenizer(examples, truncation=True)

In [ ]:
# codificar todo el dataframe
encoded_df = df_sin_indice['review_text'].map(tokenize_reviews)

# Incluir las reseñas codificadas y las etiquetas
df_merged_tmp = pd.merge(df_sin_indice, encoded_df, left_index=True, right_index=True)
cols = ["labels", "review_text_y"]
df_merged = df_merged_tmp[cols]
df_merged.rename(columns={'review_text_y': 'review_text'}, inplace=True)

<ipython-input-17-d13bdc30a586>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged.rename(columns={'review_text_y': 'review_text'}, inplace=True)


In [ ]:
# Función para actualizar el diccionario e incluir la etiqueta
def agr_label(row):
    row['review_text']['labels'] = row['labels']
    return row

df_merged_tmp2 = df_merged.apply(agr_label, axis=1)

# Se incluye un diccionario que incluye [input_ids, attention_mask, labels]
df_merged_out = df_merged_tmp2["review_text"]

In [ ]:
# Se dividen los datos en train y test
from sklearn.model_selection import train_test_split

train, test = train_test_split(df_merged_out, test_size=0.2, random_state=0)

In [ ]:
#train #26292
#test #6574
#test[7866]

In [ ]:
# Reset index
encoded_dataset_train = train.reset_index(drop=True)
# Reset index
encoded_dataset_test = test.reset_index(drop=True)

In [ ]:
encoded_dataset_train[0] #head()

{'input_ids': [0, 434, 3457, 347, 5747, 66, 332, 1711, 341, 6140, 21034, 8496, 522, 623, 332, 341, 361, 15034, 313, 332, 6008, 313, 332, 10393, 4449, 29647, 5393, 387, 1011, 45729, 342, 387, 9360, 3754, 522, 6573, 2849, 4158, 1125, 341, 563, 25041, 334, 405, 9360, 1479, 341, 683, 37662, 313, 20493, 921, 647, 683, 1659, 328, 23944, 623, 403, 689, 1521, 341, 1233, 719, 5767, 419, 1317, 14798, 405, 998, 8345, 405, 40766, 68, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': 1}

# Cargar el modelo preentrenado

In [ ]:
from transformers import AutoModelForSequenceClassification

num_labels = 2
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at BSC-TeMU/roberta-base-bne and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# De las input IDs a los hidden states

In [ ]:
outputs = model(**encoded_text)
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[-0.0262, -0.1424]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

# Definir las métricas de rendimiento

In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.4 MB/s eta 0:00:00


In [ ]:
'''
from datasets import load_metric
metric = load_metric("accuracy")
metric
'''

import evaluate
metric = evaluate.load('accuracy')
metric



EvaluationModule(name: "accuracy", module_type: "metric", features: {'predictions': Value(dtype='int32', id=None), 'references': Value(dtype='int32', id=None)}, usage: """
Args:
    predictions (`list` of `int`): Predicted labels.
    references (`list` of `int`): Ground truth labels.
    normalize (`boolean`): If set to False, returns the number of correctly classified samples. Otherwise, returns the fraction of correctly classified samples. Defaults to True.
    sample_weight (`list` of `float`): Sample weights Defaults to None.

Returns:
    accuracy (`float` or `int`): Accuracy score. Minimum possible value is 0. Maximum possible value is 1.0, or the number of examples input, if `normalize` is set to `True`.. A higher score means higher accuracy.

Examples:

    Example 1-A simple example
        >>> accuracy_metric = evaluate.load("accuracy")
        >>> results = accuracy_metric.compute(references=[0, 1, 2, 0, 1, 2], predictions=[0, 1, 1, 2, 1, 0])
        >>> print(results)
    

In [ ]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

def compute_loss(model, inputs, return_outputs=False):
    """
    Custom loss computation function.
    This function modifies the forward pass to include the labels and return the loss.
    """
    # Extract labels from inputs
    labels = inputs.get("labels")

    # Remove labels from the input dictionary to avoid unexpected behavior
    if labels is not None:
        del inputs["labels"]

    # Perform the forward pass
    outputs = model(**inputs, labels=labels)
    # Return the loss
    return (outputs.loss, outputs) if return_outputs else outputs.loss


# Afinar el modelo preentrenado

In [ ]:
from transformers import TrainingArguments

model_name = model_checkpoint.split("/")[-1]

batch_size = 16
num_train_epochs=2
num_train_samples = 26292
#train_dataset = encoded_dataset["train"].shuffle(seed=42).select(range(num_train_samples))
train_dataset = encoded_dataset_train.sample(frac=1, random_state=42).iloc[:num_train_samples]

logging_steps = len(train_dataset) // (2 * batch_size * num_train_epochs)

training_args = TrainingArguments(
    output_dir="results",
    num_train_epochs=num_train_epochs,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=logging_steps,
    push_to_hub=True,
    hub_model_id=f"{model_name}-finetuned-pcana_reviews"
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=encoded_dataset_test,
    tokenizer=tokenizer
)

<ipython-input-36-40a3145eea31>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss,Accuracy
1,0.022300,0.049263,0.984484
2,0.009700,0.084899,0.983115


TrainOutput(global_step=3288, training_loss=0.021079223978315048, metrics={'train_runtime': 1776.3, 'train_samples_per_second': 29.603, 'train_steps_per_second': 1.851, 'total_flos': 4773722544042240.0, 'train_loss': 0.021079223978315048, 'epoch': 2.0})

In [ ]:
'''
# Instead, define the loss function within the training loop
def custom_training_loop(trainer):
  """Custom training loop to incorporate the custom loss function."""
  train_dataloader = trainer.get_train_dataloader()

  for epoch in range(int(trainer.args.num_train_epochs)):
    for step, inputs in enumerate(train_dataloader):
      # ... (Your training logic here) ...

      # Calculate loss using your custom function
      loss = compute_loss(trainer.model, inputs)

      # ... (Rest of your training logic, including backpropagation) ...

# Replace the default training loop with the custom one
trainer.training_loop = custom_training_loop

# Now you can start the training process
trainer.train()
'''

'\n# Instead, define the loss function within the training loop\ndef custom_training_loop(trainer):\n  """Custom training loop to incorporate the custom loss function."""\n  train_dataloader = trainer.get_train_dataloader()\n\n  for epoch in range(int(trainer.args.num_train_epochs)):\n    for step, inputs in enumerate(train_dataloader):\n      # ... (Your training logic here) ...\n\n      # Calculate loss using your custom function\n      loss = compute_loss(trainer.model, inputs)\n\n      # ... (Rest of your training logic, including backpropagation) ...\n\n# Replace the default training loop with the custom one\ntrainer.training_loop = custom_training_loop\n\n# Now you can start the training process\ntrainer.train()\n'

# Descargue el modelo desde el Hub

In [ ]:
from transformers import pipeline

model_checkpoint = "Serchs/roberta-base-bne-finetuned-pcana_reviews-finetuned-pcana_reviews"
pipe = pipeline("sentiment-analysis", model=model_checkpoint)

config.json:   0%|          | 0.00/783 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/851k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/509k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.66M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
pipe("¡me encanta el ipad!")

[{'label': 'LABEL_1', 'score': 0.9998681545257568}]

In [ ]:
pipe("¡no me ha llegado, he reclamado y me dice que está en reparto!")

[{'label': 'LABEL_0', 'score': 0.9971786737442017}]